본격적으로 특징 벡터를 만들기에 앞서 텍스트 마이닝에서 중요하게 사용되는 TF-IDF를 사용하여 특징을 추출해보겠습니다

<br></br>
$TF-IDF(w,d) = \frac{TF(w,d)}{DF(w)}$
<br></br>

TF-IDF는 출현 빈도를 사용하여 어떤 단어 w가 문서 d내에서 얼마나 중요한지 나타내는 수치입니다. 이 수치가 높을수록 w는 d를 대표하는 성질을 띠게 된다고 볼 수도 있습니다. TF는 term frequency의 약어로, 단어의 문서내 출현한 횟수를 의미합니다. 그리고 IDF는 inverse document frequency의 약어로, 그 단어가 출현한 문서의 숫자의 역수를 의미합니다.

TF는 단어가 문서에 출현한 횟수입니다. 따라서 그 숫자가 클수록 문서에서 중요한 단어일 확률이 높습니다. 그런데 "the"와 같은 단어도 TF 값이 매우 클 것입니다. 하지만 "the"가 중요한 경우는 거의 없습니다. 이때 IDF가 필요합니다. DF는 그 단어가 출현한 문서의 숫자를 의미하므로, 그 값이 클수록 "the"와 같이 일반적으로 많이 쓰이는 단어일 가능성이 높습니다. 따라서 IDF를 구해 TF에 곱해줌으로써 "the"와 같은 단어들에 대한 패널티를 줍니다. 최종적으로 우리가 얻는 숫자는, 다른 문서들에서는 잘 나타나지 않지만 특정 문서에서만 잘 나타난 경우에 횟수가 높아지므로, 특정 문서에서 얼마나 중요한 역할을 차지하는지 나타내는 수치가 될 수 있습니다.

### TF-IDF 예제

간단하게 $분절^{tokenize}$된 여러 문서가 주어졌을때 단어들의 TF-IDF를 구하는 방법을 살펴보겠습니다. 예를 들어 서로 다른 3개의 강연 스크립트가 있다고 가정하고, 그중 일부 문장을 분절하여 TF-IDF를 구해보는 연습입니다.

먼저 특정 문서가 주어졌을때, 문서내 단어들의 출현 빈도를 세는 함수는 다음과 같이 구현할 수 있습니다.

In [1]:
import pandas as pd

def get_term_frequency(document, word_dict = None):
    if word_dict is None:
        word_dict = {}
    words = document.split()
    
    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])
        
    return pd.Series(word_dict).sort_values(ascending = False)

그리고 문서들이 주어졌을때 각 단어가 몇개의 문서에서 나타났는지 세는 함수는 다음과 같습니다.

In [2]:
def get_document_frequency(documents):
    dicts = []
    vocab = set([])
    df = {}
    
    for d in documents:
        tf = get_term_frequency(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())
        
    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1
                
    return pd.Series(df).sort_values(ascending = False)

3개의 강연 스크립트가 담긴 문서들이 각각 다음과 같은 이름으로 변수에 들어있다고 가정합니다

`doc1, doc2, doc3`

In [3]:
doc1 = '''
지능 지수 라는 말 들 어 보 셨 을 겁니다 . 여러분 의 지성 을 일컫 는 말 이 죠 . 그런데 심리 지수 란 건 뭘까요 ? 사람 들 이 특정 한 식 으로 행동 하 는 이유 에 대해 여러분 은 얼마나 알 고 계시 나요 ? 또 타인 이나 심지어 여러분 의 행동 을 예측 하 는 일 은 얼마나 잘 하 시 나요 ? 또 , 심리학 에 대해 갖춘 지식 중 에서 어느 정도 나 잘못 된 것 일까요 ? 심리학 에 관한 열 가지 신화 를 통해 잘못 된 것 들 을 알아보 도록 하 죠 . 여러분 은 한 번 쯤 들 어 보 셨 을 법 한 것 은 자신 들 의 심리학 에 대해 고려 할 때 , 거의 항상 남자 는 화성 에서 왔 고 , 여자 는 금성 에서 온 것 같 다고 합니다 . 하지만 실제로 남자 와 여자 는 얼마나 다른 걸까요 ? 이 를 알아보 기 위해 , 일단 남녀 사이 에 확실 하 게 차이 나 는 것 을 살펴보 고 심리학 적 인 성별 간 의 차이점 을 동일 한 척도 상 에서 대비 해 보 도록 하 겠 습니다 . 남자 와 여자 간 에 실제로 차이 나 는 능력 중 하나 는 그 들 이 공 을 얼마나 멀리 던질 수 있 느냐 하 는 것 입니다 . 여기 남자 들 의 데 이타 를 보 시 면 , 정상 분포 곡선 이 라는 걸 볼 수 있 습니다 . 남자 들 소수 는 정말 멀리 던지 고 , 남자 들 소수 는 멀리 던지 지 못하 지만 , 남자 들 대부분 은 평균 적 인 거리 를 던졌 습니다 . 여자 들 도 역시 비슷 한 분포 상태 를 보입니다 만 사실 남녀 사이 엔 커다란 차이 가 있 습니다 . 사실 , 평균 수준 의 남자 라면 모든 여성 중 대략 98 % 보다 더 멀리 던질 수 있 거든요 . 이 와 동일 하 게 표준 화 된 척도 상 에서 심리학 에서 말 하 는 성별 간 의 차이 를 살펴 봅시다 . 심리학자 라는 여러분 에게 말 하 길 남자 들 의 공간 지각 능력 이 여자 들 보다 뛰어나 다고 할 겁니다 . 예 를 들 어 , 지도 읽 는 능력 같 은 건데 , 맞 는 말 입니다 . 하지만 그 차이 의 정도 를 살펴봅시다 . 아주 작 죠 . 두 선 이 너무 근접 해서 거의 겹칠 정도 입니다 .
'''

doc2 = '''
최상 의 제시 유형 은 학습 자 에 좌우 되 는 것 이 아니 라 학습 해야 할 내용 에 따라 좌우 됩니다 . 예 를 들 어 여러분 이 운전 하 기 를 배울 때 실제로 몸 으로 체감 하 는 경험 없이 누군가 가 어떻게 할 지 이야기 하 는 것 을 듣 는 것 만 으로 배울 수 있 습니까 ? 연립 방정식 을 풀 어야 하 는데 종이 에 쓰 지 않 고 머리 속 에서 말 하 는 것 으로 풀 수 가 있 을까요 ? 또는 만일 여러분 이 체감 형식 의 학습 자 유형 이 라면 , 건축학 시험 을 해석 적 춤 을 이용 하 여 수정 할 수 있 을까요 ? 아니 죠 ! 배워야 할 내용 을 제시 된 유형 에 맞추 어야 합니다 , 당신 에게 맞추 는 게 아니 라요 . 여러분 들 상당수 가 " A " 급 의 우등 생 이 라는 걸 아 는데 , 조만간 중등 학력 인증 시험 ( GCSE ) 결과 를 받 게 되 시 겠 네요 . 그런데 , 만일 , 여러분 들 이 희망 했 던 성적 을 받 지 못하 게 된다 해도 여러분 들 의 학습 방식 을 탓 해서 는 안 되 는 겁니다 . 여러분 이 비난 할 수 있 는 한 가지 는 바로 유전자 입니다 . 이건 최근 에 런던 대학교 ( UCL ) 에서 수행 했 던 연구 결과 는 여러 학생 들 과 그 들 의 중등 학력 인증 시험 결과 사이 의 차이 중 58 % 는 유전 적 인 요인 으로 좁혀졌 습니다 . 매우 정밀 한 수치 처럼 들립니다 . 그러면 어떻게 알 수 있 을까요 ? 유전 적 요인 과 환경 적 요인 의 상대 적 기여 도 를 알 고 싶 을 때 우리 가 사용 할 수 있 는 방식 은 바로 쌍둥이 연구 입니다 . 일 란 성 쌍생아 의 경우 환경 적 요인 과 유전 적 요인 모두 를 100 % 똑같이 공유 하 게 되 지만 이란 성 쌍생아 의 경우 는 100 % 동일 한 환경 을 공유 하 지만 유전자 의 경우 여타 의 형제자매 들 처럼 50 % 만 공유 하 게 됩니다 . 따라서 일 란 성 쌍둥이 와 이란 성 쌍둥이 사이 의 인증 시험 결과 가 얼마나 비슷 한지 비교 해 보 고 여기 에 약간 의 수학 적 계산 을 더하 게 되 면 그 수행 능력 의 차이 중 어느 정도 가 환경 적 요인 의 탓 이 고 어느 정도 가 유전자 탓 인지 를 알 수 있 게 됩니다 .
'''

doc3 = '''
그러나 이 이야기 는 세 가지 이유 로 인해 신화 입니다 . 첫째 , 가장 중요 한 건 실험실 가운 은 흰색 이 아니 라 회색 이 었 다 라는 점 이 죠 . 둘째 , 참 여자 들 은 실험 하 기 전 에 와 참여 자 들 이 걱정 을 표현 할 때 마다 상기 시키 는 말 을 들 었 는데 , 전기 충격 이 고통 스럽 기 는 하 지만 , 치명 적 이 지 는 않 으며 실제로 영구 적 인 손상 을 남기 는 일 은 없 을 거 라는 것 이 었 습니다 . 셋째 , 참 여자 들 은 단지 가운 을 입 은 사람 이 시켜 전기 충격 을 주지 는 않 았 죠 . 실험 이 끝나 고 그 들 의 인터뷰 를 했 을 때 모든 참여 자 들 은 강한 신념 을 밝혔 는데 , ' 학습 과 처벌 ' 연구 가 과학 적 으로 가치 있 는 목적 을 수행 했 기 때문 에 비록 동료 참여 자 들 에게 가해진 순간 적 인 불편 함 에 반해서 과학 을 위해서 오래 남 을 성과 를 얻 을 것 이 라고 말 이 죠 . 그러 다 보 니 제 가 이야기 를 한 지 벌써 12 분 이 되 었 습니다 . 여러분 들 중 에 는 아마 거기 앉 아서 제 이야기 를 들으시는 동안 저 의 말투 와 몸짓 을 분석 하 면서 제 가 말 하 는 어떤 것 을 인지 해야 할까 해결 하 려고 하 셨 을 겁니다 , 제 가 진실 을 이야기 하 는 지 , 또는 거짓말 을 하 고 있 는 것 인지 말 이 죠 . 만일 그러 셨 다면 , 아마 지금 쯤 완전히 실패 하 셨 을 겁니다 . 왜냐하면 우리 모두 가 사람 이 말 하 는 패턴 과 몸짓 으로 도 거짓말 여부 를 알아내 는 것 이 가능 하 다고 생각 하 지만 , 오랜 세월 수백 회 에 걸쳐 행해진 실제 심리 검사 의 결과 를 보 면 우리 들 모두 는 , 심지어 경찰관 이나 탐정 들 을 포함 해서 도 기본 적 으로 몸짓 과 언어 적 패턴 으로 거짓말 을 탐지 하 는 것 은 운 에 맞 길 수 밖 에 는 없 는 것 입니다 . 흥미 롭 게 도 한 가지 예외 가 있 는데요 : 실종 된 친척 을 찾 아 달 라고 호소 하 는 TV 홍보 입니다 .
'''

그러면 TF-IDF를 계산하는 최종함수는 다음과 같이 구현할 수 있습니다.

In [4]:
def get_tfidf(docs):
    vocab = {}
    tfs = []
    
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]
        
    df = get_document_frequency(docs)
    
    from operator import itemgetter
    import numpy as np
    
    stats = []
    
    for word, freq in vocab.items():
        tfidfs = []
        
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
                
            else:
                tfidfs += [0]
                
        stats.append((word, freq, *tfidfs, max(tfidfs)))
        
    return pd.DataFrame(stats, columns = ("words",
                                          "frequency",
                                          "doc1",
                                          "doc2",
                                          "doc3",
                                          "max")).sort_values("max", ascending = False)

그 결과는 다음과 같습니다.

In [5]:
get_tfidf([doc1, doc2, doc3]).head(10)

,words,frequency,doc1,doc2,doc3,max
23,남자,9,9.887511,0.000000,0.000000,9.887511
37,요인,6,0.000000,6.591674,0.000000,6.591674
52,심리학,5,5.493061,0.000000,0.000000,5.493061
66,성,4,0.000000,4.394449,0.000000,4.394449
57,었,4,0.000000,0.000000,4.394449,4.394449
62,환경,4,0.000000,4.394449,0.000000,4.394449
63,시험,4,0.000000,4.394449,0.000000,4.394449
70,제,4,0.000000,0.000000,4.394449,4.394449
69,멀리,4,4.394449,0.000000,0.000000,4.394449
81,참여,3,0.000000,0.000000,3.295837,3.295837


첫번째 스크립트에서 가장 중요한 단어는 "남자"이고 두번재 스크립트에서 가장 중요한 단어는 "요인" 입니다.